### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 7.7.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run baseline_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    beta = params
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': np.ndarray.tolist(trial_sequence), 'Choice' : choices, 'Trustee_Response': responses, 'Beta': [beta] * tn *3 }    
    return data

In [8]:
#sim_data(15,2)

### Create set of parameters
- distribution of beta parameters modeled following Cutler et al., 2021

In [9]:
#betas = [x * 2 for x in range(0,11)]
betas = gamma.rvs(a = 1.2, scale = 5, size=150)
#print(betas)

### Simulate data

In [10]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Beta', 'Subject'])
for p in range(len(betas)):
    dt = sim_data(15, betas[p])
    dt['Subject'] = [p + 1] * 45
    dt = pd.DataFrame(dt)
    data = pd.concat([data, dt])
    
data.to_csv(path_or_buf = '../../output/simulation/sim_baseline_model_data.csv', index = False)

In [11]:
# df = data[data['Subject']==120]
# df

### Pull in existing data


In [12]:
dt = pd.read_csv('../../output/simulation/sim_baseline_model_data.csv')

### New function to fit model to multiple subjects

In [13]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'beta', 'llh'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==(sub)]
        dt = df.to_dict()
        params = model_fit(dt)
        line = {'Subject': sub, 'beta': params.x[0], 'llh':params.fun}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [14]:
precover = model_fit_mult_subject(data)

Number of Subs:  150
Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31
Subject:  32
Subject:  33
Subject:  34
Subject:  35
Subject:  36
Subject:  37
Subject:  38
Subject:  39
Subject:  40
Subject:  41
Subject:  42
Subject:  43
Subject:  44
Subject:  45
Subject:  46
Subject:  47
Subject:  48
Subject:  49
Subject:  50
Subject:  51
Subject:  52
Subject:  53
Subject:  54
Subject:  55
Subject:  56
Subject:  57
Subject:  58
Subject:  59
Subject:  60
Subject:  61
Subject:  62
Subject:  63
Subject:  64
Subject:  65
Subject:  66
Subject:  67
Subject:  68
Subject:  69
Subject:  70
Subject:  71
Subject:  72
Subject:  73
Subject:  74
Subject:  75
Subject:  76


In [15]:
print(precover)

     Subject       beta       llh
0        1.0   6.547165  8.317766
1        2.0   2.754395  8.317766
2        3.0   9.271938  8.317766
3        4.0   3.970354  8.317766
4        5.0  17.956468  8.317766
5        6.0   5.354607  8.317766
6        7.0   6.520019  8.317766
7        8.0   4.544303  8.317766
8        9.0  17.109594  8.317766
9       10.0   9.893541  8.317766
10      11.0   1.308356  8.317766
11      12.0   4.061079  8.317766
12      13.0   8.807370  8.317766
13      14.0  18.443884  8.317766
14      15.0   1.022566  8.317766
15      16.0   8.895556  8.317766
16      17.0  10.467936  8.317766
17      18.0   4.220845  8.317766
18      19.0   7.241422  8.317766
19      20.0  18.035552  8.317766
20      21.0  19.258242  8.317766
21      22.0  17.718937  8.317766
22      23.0   1.292520  8.317766
23      24.0   5.650613  8.317766
24      25.0  10.305186  8.317766
25      26.0   5.938151  8.317766
26      27.0   3.071857  8.317766
27      28.0  16.305170  8.317766
28      29.0  

In [16]:
precover.to_csv(path_or_buf = '../../output/simulation/sim_baseline_model_fits.csv', index = False)